# RAG - No vector database

In [4]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");

record Car (string make, string model, string basemodel, string color, int year, string condition, int mileage, int price);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [5]:
async Task<string> process_competion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Render an Scriban template

In [6]:
string RenderTemplate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

### Prepare the Prompt

In [2]:
var prompt_template = @"Using only the best answers, answer the following question: 
{{input}}


Text: ###
1. CRISPR (Clustered Regularly Interspaced Short Palindromic Repeats) is a revolutionary gene-editing tool that allows precise modification of an organism's DNA. It can be used to correct genetic defects, create genetically modified organisms, study gene functions, and potentially treat various diseases by editing the DNA sequence.

2. Blockchain is a decentralized, distributed ledger technology. It ensures security through cryptographic techniques and consensus protocols. Each block in the chain contains a cryptographic hash of the previous block, making it tamper-evident. Transactions are validated by multiple participants (nodes) in the network, enhancing security and preventing unauthorized changes.

3. Quantum computing leverages the principles of quantum mechanics to perform computations using quantum bits (qubits), which can exist in multiple states simultaneously, unlike classical bits that are either 0 or 1. This ability for qubits to exist in superposition allows quantum computers to solve certain complex problems much faster than classical computers.

4. Machine learning is a subset of artificial intelligence where systems learn from data and improve their performance over time without being explicitly programmed. Its applications range from recommendation systems in online platforms, fraud detection in financial transactions, autonomous vehicles, natural language processing, to medical diagnostics and image recognition.

5. Self-driving cars use a combination of sensors such as cameras, lidar, radar, and GPS to perceive the environment. These sensors collect data, which is processed by sophisticated algorithms to create a real-time map of the car's surroundings. Decision-making algorithms then analyze this data to make decisions about steering, acceleration, and braking, allowing the vehicle to navigate and operate autonomously.

6. NFTs are a specialized application of blockchain technology, utilizing its security and decentralization to authenticate and establish ownership of digital assets that are unique and non-interchangeable.

###

Output format: ###
Question: ''
Answer: ''
###


Use only the provided text. Provide the original question and the answer. If the question cannot be answered from the text, answer ""I don't know.""";

### Anser the questions

In [7]:
// Execute the SK function
var input="What is CRISPR?";
var prompt = RenderTemplate(prompt_template, new {input});
Console.WriteLine(await process_competion(prompt, 100));

Question: 'What is CRISPR?'
Answer: 'CRISPR (Clustered Regularly Interspaced Short Palindromic Repeats) is a revolutionary gene-editing tool that allows precise modification of an organism's DNA. It can be used to correct genetic defects, create genetically modified organisms, study gene functions, and potentially treat various diseases by editing the DNA sequence.'


In [8]:
input="What is an NFT?";
var prompt = RenderTemplate(prompt_template, new {input});
Console.WriteLine(await process_competion(prompt, 100));


Question: 'What is an NFT?'
Answer: 'NFTs are a specialized application of blockchain technology, utilizing its security and decentralization to authenticate and establish ownership of digital assets that are unique and non-interchangeable.'


In [9]:
input="What is the speed of light?";
var prompt = RenderTemplate(prompt_template, new {input});
Console.WriteLine(await process_competion(prompt, 100));

Question: 'What is the speed of light?'
Answer: 'I don't know.'
